In [1]:
import os
import numpy
from tensorflow import keras
import librosa
from matplotlib import pyplot


def get_mfcc(wav_file_path):
    y, sr = librosa.load(wav_file_path, offset=0, duration=30)
    mfcc = numpy.array(librosa.feature.mfcc(y=y, sr=sr))
    return mfcc


def get_melspectrogram(wav_file_path):
    y, sr = librosa.load(wav_file_path, offset=0, duration=30)
    melspectrogram = numpy.array(librosa.feature.melspectrogram(y=y, sr=sr))
    return melspectrogram


def get_chroma_vector(wav_file_path):
    y, sr = librosa.load(wav_file_path)
    chroma = numpy.array(librosa.feature.chroma_stft(y=y, sr=sr))
    return chroma


def get_tonnetz(wav_file_path):
    y, sr = librosa.load(wav_file_path)
    tonnetz = numpy.array(librosa.feature.tonnetz(y=y, sr=sr))
    return tonnetz


def get_feature(file_path):
    # Extracting MFCC feature
    mfcc = get_mfcc(file_path)
    mfcc_mean = mfcc.mean(axis=1)
    mfcc_min = mfcc.min(axis=1)
    mfcc_max = mfcc.max(axis=1)
    mfcc_feature = numpy.concatenate((mfcc_mean, mfcc_min, mfcc_max))

    # Extracting Mel Spectrogram feature
    melspectrogram = get_melspectrogram(file_path)
    melspectrogram_mean = melspectrogram.mean(axis=1)
    melspectrogram_min = melspectrogram.min(axis=1)
    melspectrogram_max = melspectrogram.max(axis=1)
    melspectrogram_feature = numpy.concatenate(
        (melspectrogram_mean, melspectrogram_min, melspectrogram_max))

    # Extracting chroma vector feature
    chroma = get_chroma_vector(file_path)
    chroma_mean = chroma.mean(axis=1)
    chroma_min = chroma.min(axis=1)
    chroma_max = chroma.max(axis=1)
    chroma_feature = numpy.concatenate((chroma_mean, chroma_min, chroma_max))

    # Extracting tonnetz feature
    tntz = get_tonnetz(file_path)
    tntz_mean = tntz.mean(axis=1)
    tntz_min = tntz.min(axis=1)
    tntz_max = tntz.max(axis=1)
    tntz_feature = numpy.concatenate((tntz_mean, tntz_min, tntz_max))

    feature = numpy.concatenate(
        (chroma_feature, melspectrogram_feature, mfcc_feature, tntz_feature))
    return feature

directory = 'dataset'
genres = ['pop','hiphop','metal']
features = []
labels = []
for genre in genres:
    print("Calculating features for genre : " + genre)
    for file in os.listdir(directory+"/"+genre):
        file_path = directory+"/"+genre+"/"+file

        features.append(get_feature(file_path))
        label = genres.index(genre)
        labels.append(label)
        
permutations = numpy.random.permutation(300)
features = numpy.array(features)[permutations]
labels = numpy.array(labels)[permutations]

features_train = features[0:180]
labels_train = labels[0:180]

features_val = features[180:240]
labels_val = labels[180:240]

features_test = features[240:300]
labels_test = labels[240:300]

inputs = keras.Input(shape=(498), name="feature")
x = keras.layers.Dense(300, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(200, activation="relu", name="dense_2")(x)
x = keras.layers.Dense(100, activation="relu", name="dense_3")(x)
outputs = keras.layers.Dense(3, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    # Optimizer
    optimizer=keras.optimizers.RMSprop(),
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
model.fit(x=features_train.tolist(),y=labels_train.tolist(),verbose=1,validation_data=(features_val.tolist() , labels_val.tolist()), epochs=200)

score = model.evaluate(x=features_test.tolist(),y=labels_test.tolist(), verbose=0)
print('Accuracy : ' + str(score[1]*100) + '%')


Calculating features for genre : pop
Calculating features for genre : hiphop
Calculating features for genre : metal
Epoch 1/200
6/6 [==============================] - 1s 74ms/step - loss: 118.1270 - sparse_categorical_accuracy: 0.4389 - val_loss: 49.0798 - val_sparse_categorical_accuracy: 0.4667
Epoch 2/200
6/6 [==============================] - 0s 22ms/step - loss: 24.9795 - sparse_categorical_accuracy: 0.4667 - val_loss: 59.8308 - val_sparse_categorical_accuracy: 0.3333
Epoch 3/200
6/6 [==============================] - 0s 22ms/step - loss: 18.9585 - sparse_categorical_accuracy: 0.5722 - val_loss: 35.6460 - val_sparse_categorical_accuracy: 0.4000
Epoch 4/200
6/6 [==============================] - 0s 21ms/step - loss: 10.3532 - sparse_categorical_accuracy: 0.6667 - val_loss: 9.3291 - val_sparse_categorical_accuracy: 0.5000
Epoch 5/200
6/6 [==============================] - 0s 29ms/step - loss: 7.6410 - sparse_categorical_accuracy: 0.7167 - val_loss: 11.9397 - val_sparse_categorical_ac

In [4]:
file_path = "audio_sample_30s_metal.wav"
feature = get_feature(file_path)
y = model.predict(feature.reshape(1,498))
ind = numpy.argmax(y)
print('Predicted Genre: '+genres[ind])

1/1 [==============================] - 0s 16ms/step
Predicted Genre: metal
